This notebook uses the `NeuralNetworkBuilder` API from CoreML to manually create a neural network. Only the `convolution` and `fully connected` layers are trainable at the moment.

In [1]:
import numpy as np
from numpy.random import rand

from coremltools.models import datatypes
from coremltools.models.neural_network import NeuralNetworkBuilder
from coremltools.models.utils import save_spec

XGBoost version 1.7.1 has not been tested with coremltools. You may run into unexpected errors. XGBoost 1.4.2 is the most recent version that has been tested.
2023-11-27 14:22:52.659307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [34]:
input_dim = (1, 28, 28)
output_dim = (10,)

In [35]:
# Specify input and output features
input_features = [("input", datatypes.Array(*input_dim))]
output_features = [("prediction", datatypes.Array(*output_dim))]

In [36]:
builder = NeuralNetworkBuilder(input_features, output_features)

W_conv2d_1 = rand(1,32,3,3)
W_conv2d_2 = rand(32,64,3,3)
W_conv2d_3 = rand(64,64,3,3)

W_fc_1 = rand(576,300)
W_fc_2 = rand(300,10)

builder.add_convolution(name="conv2d_1",
                        kernel_channels=1,
                        output_channels=32,
                        height=3,
                        width=3,
                        stride_height=1,
                        stride_width=1,
                        border_mode='valid',
                        groups=1,
                        W=W_conv2d_1,
                        b=None,
                        has_bias=False,
                        input_name="input",
                        output_name="conv2d_1_output")

builder.add_activation(name="relu_1",
                       non_linearity="RELU",
                       input_name="conv2d_1_output",
                       output_name="conv2d_1_relu")

builder.add_pooling(name="pool2d_1",
                    height=2,
                    width=2,
                    stride_height=2,
                    stride_width=2,
                    layer_type="MAX",
                    padding_type="VALID",
                    input_name="conv2d_1_relu",
                    output_name="pool2d_1_out")

builder.add_convolution(name="conv2d_2",
                        kernel_channels=32,
                        output_channels=64,
                        height=3,
                        width=3,
                        stride_height=1,
                        stride_width=1,
                        border_mode='valid',
                        groups=1,
                        W=W_conv2d_2,
                        b=None,
                        has_bias=False,
                        input_name="pool2d_1_out",`
                        output_name="conv2d_2_output")

builder.add_activation(name="relu_2",
                       non_linearity="RELU",
                       input_name="conv2d_2_output",
                       output_name="conv2d_2_relu")

builder.add_pooling(name="pool2d_2",
                    height=2,
                    width=2,
                    stride_height=2,
                    stride_width=2,
                    layer_type="MAX",
                    padding_type="VALID",
                    input_name="conv2d_2_relu",
                    output_name="pool2d_2_out")

builder.add_convolution(name="conv2d_3",
                        kernel_channels=64,
                        output_channels=64,
                        height=3,
                        width=3,
                        stride_height=1,
                        stride_width=1,
                        border_mode='valid',
                        groups=1,
                        W=W_conv2d_3,
                        b=None,
                        has_bias=False,
                        input_name="pool2d_2_out",
                        output_name="conv2d_3_output")

builder.add_activation(name="relu_3",
                       non_linearity="RELU",
                       input_name="conv2d_3_output",
                       output_name="conv2d_3_relu")

builder.add_flatten(name="flatten",
                    mode=0,
                    input_name="conv2d_3_relu",
                    output_name="flatten_out")

builder.add_inner_product(name="dense1",
                          W=W_fc_1,
                          b=None,
                          has_bias=False,
                          input_channels=576,
                          output_channels=300,
                          input_name="flatten_out",
                          output_name="dense1_out")

builder.add_activation(name="relu_4",
                       non_linearity="RELU",
                       input_name="dense1_out",
                       output_name="dense1_relu_out")

builder.add_inner_product(name="dense2",
                          W=W_fc_2,
                          b=None,
                          has_bias=False,
                          input_channels=300,
                          output_channels=10,
                          input_name="dense1_relu_out",
                          output_name="dense2_out")

builder.add_softmax(name="softmax",
                    input_name="dense2_out",
                    output_name="prediction")

builder.layers

['conv2d_1',
 'relu_1',
 'pool2d_1',
 'conv2d_2',
 'relu_2',
 'pool2d_2',
 'conv2d_3',
 'relu_3',
 'flatten',
 'dense1',
 'relu_4',
 'dense2',
 'softmax']

In [37]:
# only conv2d and inner product layers are updatable

builder.make_updatable(["conv2d_1", "conv2d_2", "conv2d_3", "dense1", "dense2"])

In [38]:
builder.set_categorical_cross_entropy_loss(name='lossLayer', input='prediction')

Now adding input prediction_true as target for categorical cross-entropy loss layer.


In [39]:
from coremltools.models.neural_network import SgdParams
builder.set_sgd_optimizer(SgdParams(lr=0.01, batch=32))

In [40]:
builder.set_epochs(10)

In [41]:
!rm -r cnn.mlpackage

In [42]:
save_spec(builder.spec, "cnn.mlpackage")

This cell below is used as a "test" script to verify the shapes of the output vectors from each operation.

In [15]:
# import torch

# x = torch.randn(1, 1, 28, 28)
# print(x.shape)

# conv1 = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)(x)
# print(conv1.shape)

# pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(conv1)
# print(pool1.shape)

# conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)(pool1)
# print(conv2.shape)

# pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(conv2)
# print(pool2.shape)

# conv3 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)(pool2)
# print(conv3.shape)

# flatten = torch.nn.Flatten()(conv3)
# print(flatten.shape)

# fc = torch.nn.Linear(3136,500)(flatten)
# print(fc.shape)

# relu_fc = torch.nn.ReLU()(fc)
# print(relu_fc.shape)

# fc2 = torch.nn.Linear(500,10)(relu_fc)
# print(fc2.shape)

# result = torch.nn.Softmax(dim=0)(fc2)
# result, result.shape